In [1]:

def get_ladder_json(word_length):
    assert word_length >= 3 and word_length <=6

    import requests
    import json
    r = requests.get(f'https://raw.githubusercontent.com/shaunhegarty/word-ladder/master/wordladder/resources/ladder-common-{word_length}.json')
    data = json.loads(r.content.decode('utf-8'))
    return data

data = get_ladder_json(3)

In [2]:
import sys
sys.path.append('/workspace/')

from sqlalchemy.orm import Session

from anagrammer import models
from anagrammer.database import engine

models.Base.metadata.create_all(bind=engine)

In [ ]:
with Session(engine) as session:
    results = session.query(models.Dictionary).filter(models.Dictionary.frequency > 50000).all()
    print(results)

In [3]:
from sqlalchemy import func

def sort_word_pair(pair):
    return '-'.join(sorted(pair.split('-')))

def get_word_scores():

    with Session(engine) as session:
        results = (
            session.query(
                models.Dictionary,
                func.rank().over(order_by=models.Dictionary.frequency.desc()),
            )
            .filter(models.Dictionary.dictionary == "common")
            .all()
        )
        word_scores = {word.word: rank for word, rank in results}
    return word_scores

def ladder_difficulty(ladder, word_scores):
    ladder_score = 0
    
    for word in ladder:
        ladder_score += word_scores.get(word)
        
    return ladder_score

def get_hardest_word(ladder, word_scores):
    hardest_word_score = 0
    hardest_word = None
    for word in ladder:
        word_score = word_scores.get(word)
        if word_score > hardest_word_score:
            hardest_word_score = word_score
            hardest_word = word
    return hardest_word, hardest_word_score

In [ ]:
from sqlalchemy.dialects.postgresql import insert

with Session(engine) as session:
    # Populate Word Score
    values = []
    for word, word_score in get_word_scores.items():
        values.append({'word': word, 'dictionary': 'common', 'score': word_score})
    statement = (
        insert(models.WordScore).values(values).on_conflict_do_nothing()
    )
    session.execute(statement)
    session.commit()

In [4]:
from sqlalchemy.dialects.postgresql import insert

word_scores = get_word_scores()

for word_length in range(3, 7):
    data = get_ladder_json(word_length=word_length)
    unique_ladder_keys = {sort_word_pair(pair) for pair in data.keys()}

    print(f'{len(unique_ladder_keys)} ladder keys')
    values = []
    for key in list(unique_ladder_keys):
        ladder_list = data.get(key)
        for index, ladder in enumerate(ladder_list):
            hardest_word, hardest_word_score = get_hardest_word(ladder, word_scores)
            values.append({'pair': key,
                'dictionary': 'common',
                'chain': ','.join(ladder),
                "length": len(ladder),
                'difficulty': ladder_difficulty(ladder, word_scores),
                'hardest_word': hardest_word,
                'hardest_word_score': hardest_word_score,
                'variations': len(ladder_list),
                'variant': index + 1, })
    print(f'Updating Database with {len(values)} records')
    with Session(engine) as session:
        statement = (
            insert(models.Ladder).values(values).on_conflict_do_nothing()
        )
        session.execute(statement)
        session.commit()

3637 ladder keys
Updating Database with 18078 records
3015 ladder keys
Updating Database with 23748 records
2166 ladder keys
Updating Database with 12056 records
481 ladder keys
Updating Database with 5665 records


In [9]:
with Session(engine) as session:
    results = session.query(models.Ladder).filter(
            models.Ladder.length == 7, 
            models.Ladder.difficulty < 60000,
        ).order_by(models.Ladder.hardest_word_score.desc()).all()
    for r in results:
        print(r, r.hardest_word)

five-year: [five,file,fill,fell,feal,fear,year]. 1 of 2. Difficulty: 58331. feal
head-much: [head,heat,seat,sect,sech,such,much]. 3 of 3. Difficulty: 58841. sech
road-they: [road,read,reed,reen,teen,then,they]. 7 of 10. Difficulty: 55245. reen
film-very: [film,firm,farm,marm,mary,vary,very]. 8 of 11. Difficulty: 51728. marm
been-film: [been,peen,peel,feel,fell,fill,film]. 5 of 26. Difficulty: 57183. peen
high-more: [high,hish,wish,wise,wire,wore,more]. 17 of 18. Difficulty: 58594. hish
high-name: [high,hish,hash,cash,case,came,name]. 8 of 9. Difficulty: 56835. hish
game-high: [game,came,case,cash,hash,hish,high]. 10 of 12. Difficulty: 56856. hish
high-rock: [high,hish,dish,disk,dick,dock,rock]. 14 of 18. Difficulty: 59999. hish
high-rock: [high,hish,dish,disk,dick,rick,rock]. 5 of 18. Difficulty: 59070. hish
high-work: [high,hish,wish,wise,wire,wore,work]. 1 of 2. Difficulty: 58649. hish
high-role: [high,hish,wish,wise,rise,rose,role]. 16 of 26. Difficulty: 54659. hish
high-made: [high